In [1]:
import re
import csv
import sys
import json
import math
import spacy
import textacy
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from taxonerd import TaxoNERD
from fastcoref import spacy_component
from itertools import permutations, combinations
from spacy.matcher import Matcher, DependencyMatcher, PhraseMatcher
%run "./Main.ipynb"

C:\Users\lbeln\anaconda3\envs\3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def power_set(pool, current=None, result=None):
    if current not in result:
        result.append(current)

    for item in pool:
        if item not in current:
            power_set(pool, current={*current, item}, result=result)

    return result

def interleave(pool, current=None, used=None, skip=None, result=None):    
    if current and current not in result:
        result.append(current)
    
    for i in range(len(pool)):
        if i == skip:
            continue
        
        for j in range(len(pool[i])):
            if (i, j) in used:
                continue
            
            interleave(
                pool, 
                used=[*used, (i, j)],
                current=[*current, pool[i][j]],
                skip=i, 
                result=result
            )

    return result

In [3]:
class Node:
    def __init__(self, main, is_action=False, tokens=None):
        self.main = main
        self.tokens = tokens or []
        self.is_action = is_action
        self.all_tokens = []

    

    def start(self):
        i = math.inf
        for token in self.tokens:
            i = min(i, token.i)
        return i


    
    def end(self):
        i = -math.inf
        for token in self.tokens:
            i = max(i, token.i)
        return i

    
    
    def get_tokens(self):
        return self.tokens


    
    def get_all_tokens(self):
        return [*self.tokens, *self.all_tokens]


    
    def get_traits(self):
        tokens = set([*self.tokens, *self.all_tokens])
        tokens = tokens & set(self.main.trait.tokens)
        return tokens


    
    def get_species(self):
        tokens = set([*self.tokens, *self.all_tokens])
        tokens = tokens & set(self.main.species.tokens)
        return tokens


    
    def copy(self):
        node = Node(self.main)
        node.tokens = [*self.tokens]
        node.is_action = self.is_action
        node.all_tokens = [*self.all_tokens]
        return node
    
    
    
    def __str__(self):
        tokens = [*self.tokens, *self.all_tokens]
        tokens = list(set(tokens))
        tokens = sorted(tokens, key=lambda token: token.i)
        return f"{','.join([t.text for t in tokens])}"

    
    
    @staticmethod
    def extend_token_coref(main, all_tokens):
        i = 0
        size = len(all_tokens)
        while i < size:
            token = all_tokens[i]
            if token.pos_ == "PRON":
                all_tokens = [*main.coref_map.get(token, [token])]
                break
            i += 1
        return all_tokens
    

    
    @staticmethod
    def extend_token_unit(main, all_tokens):
        i = 0
        size = len(all_tokens)
        unit_map_items = main.units.unit_map.items()
        while i < size:
            token = all_tokens[i]
            for unit_bound, unit in unit_map_items:
                token_in_unit = unit_bound[0] <= token.i <= unit_bound[1]
                unit_is_valid = unit.label_has([Unit.ITEM])
                if token_in_unit and unit_is_valid:
                    all_tokens = [*main.sp_doc[unit_bound[0]:unit_bound[1]+1]]
                    break
            i += 1
        return all_tokens
    
    
    
    @staticmethod
    def extend_token_entity(main, all_tokens):
        i = 0
        size = len(all_tokens)
        while i < size:
            token = all_tokens[i]
            if token in main.entity_map:
                all_tokens.extend([*main.entity_map[token]])
            i += 1
        return all_tokens
    
    
    
    @staticmethod
    def extend_token_noun_chunk(main, all_tokens):
        i = 0
        size = len(all_tokens)
        while i < size:
            token = all_tokens[i]
            if token in main.noun_chunk_map:
                all_tokens.extend([*main.noun_chunk_map[token]])
            i += 1
        return all_tokens
    
    
    
    @staticmethod
    def extend_token(main, token):
        # print("Extend Token")
        all_tokens = [token]

        # print(f"Initial: {all_tokens}")
        
        all_tokens = Node.extend_token_coref(main, all_tokens)
        # print(f"After Coref: {all_tokens}")
        all_tokens = Node.extend_token_unit(main, all_tokens)
        # print(f"After Unit: {all_tokens}")
        all_tokens = Node.extend_token_entity(main, all_tokens)
        # print(f"After Entity: {all_tokens}")
        all_tokens = Node.extend_token_noun_chunk(main, all_tokens)
        # print(f"After Noun Chunk: {all_tokens}")
        
        all_tokens = list(set(all_tokens))
        all_tokens = sorted(all_tokens, key=lambda token: token.i)
        
        return all_tokens

In [4]:
class Event:
    def __init__(self, main):
        self.main = main
        self.order = []

    
    
    def start(self):
        i = math.inf
        for item in self.order:
            i = min(i, item.start())
        return i


    
    def end(self):
        i = -math.inf
        for item in self.order:
            i = max(i, item.end())
        return i



    def sent_start(self):
        start = self.start()
        sents = self.main.sp_doc.sents
        for i, sent in enumerate(sents):
            if sent.start <= start < sent.end:
                return i
        return -1


    
    def copy(self):
        event = Event(self.main)
        event.order = []
        for item in self.order:
            event.order.append(item.copy())
        return event


    
    def get_tokens(self):
        ret = []
        for item in self.order:
            ret.extend(item.get_tokens())
        return ret


    
    def get_all_tokens(self):
        ret = []
        for item in self.order:
            ret.extend(item.get_all_tokens())
        return ret


    
    def get_species(self):
        ret = []
        for item in self.order:
            ret.extend(item.get_species())
        return ret


    
    def get_traits(self):
        ret = []
        for item in self.order:
            ret.extend(item.get_traits())
        return ret
    

    
    def attach_event(self, event, del_end=False):
        if del_end:
            self.order.pop()
        self.order.extend(event.order)
        return self
    

    
    def __str__(self):
        ret = ""

        i = 0
        size = len(self.order)
        while i < size:
            ret += f"({self.order[i]})"
            if i != size - 1:
                ret += "->"
            i += 1
        
        return ret

In [43]:
class EventManager:
    def __init__(self, main):
        self.main = main
        self.events = []


    
    def fix_triple_tokens(self, s_node, v_node, o_node):
        # print("\tBefore")
        # print(f"\t\t{s_node.tokens}")
        # print(f"\t\t{v_node.tokens}")
        # print(f"\t\t{o_node.tokens}")
        
        # Subject Tokens Transferred to Object
        s_transfer_tokens = []
        for token in s_node.tokens:
            if token.pos_ == "VERB":
                s_transfer_tokens.append(token)
        
        s_node.tokens = [t for t in s_node.tokens if t not in s_transfer_tokens]
        o_node.tokens.extend(s_transfer_tokens)
        
        # Object Tokens Transferred to Verb
        o_transfer_tokens = []
        for token in o_node.tokens:
            if token in main.cause.tokens or token in main.change.tokens:
                o_transfer_tokens.append(token)
        
        o_node.tokens = [t for t in o_node.tokens if t not in o_transfer_tokens]
        v_node.tokens.extend(o_transfer_tokens)

        # print("\tAfter")
        # print(f"\t\t{s_node.tokens}")
        # print(f"\t\t{v_node.tokens}")
        # print(f"\t\t{o_node.tokens}")

        return (s_node, v_node, o_node)


    
    def fix_triple_direction(self, s_node, v_node, o_node):
        v_node_tokens = sorted(v_node.tokens, key=lambda t: t.i)
        v_node_tokens_l = v_node_tokens[0]
        v_node_tokens_r = v_node_tokens[-1]
        
        aux = v_node_tokens_l.nbor(-1) and v_node_tokens_l.nbor(-1).pos_ == "AUX"
        adp = v_node_tokens_r.nbor(1) and v_node_tokens_r.nbor(1).pos_ == "ADP"
        
        if not aux and not adp:
            return (s_node, v_node, o_node)
        return (o_node, v_node, s_node)


    
    def fix_triple_all_tokens(self, s_node, v_node, o_node):
        s_node.all_tokens = flatten([Node.extend_token(main, t) for t in s_node.tokens])
        o_node.all_tokens = flatten([Node.extend_token(main, t) for t in o_node.tokens])
        return (s_node, v_node, o_node)


    
    def push_bound_l(self, bound, min_bound):
        print("Push L")
        return self.push_bound(
            bound, 
            reverse=True, 
            unit_condition_fnc=lambda unit: unit.l >= min_bound,
            update_bound_fnc=lambda bound, unit: min(bound, unit.l)
        )


    
    def push_bound_r(self, bound, max_bound):
        print("Push R")
        return self.push_bound(
            bound, 
            reverse=False, 
            # unit_condition_fnc=lambda unit: unit.r <= max_bound,
            unit_condition_fnc=lambda unit: True,
            update_bound_fnc=lambda bound, unit: max(bound, unit.r)
        )


    
    def push_bound(self, bound, reverse=False, unit_condition_fnc=None, update_bound_fnc=None):
        units = self.main.units.units_at_i(bound)
        if not units:
            return bound

        start = False
        unit_map_vals = list(main.units.unit_map.values())
        unit_map_vals = sorted(unit_map_vals, key=lambda u: (u.l, u.r), reverse=reverse)

        i = 0
        while i < len(unit_map_vals):
            unit = unit_map_vals[i]
            print(f"Unit: {unit}")

            # Units must be in the same sentence as bound.
            if units[0].sent_start() != unit.sent_start():
                i += 1
                print("Not Same Sent")
                continue

            if unit not in units:
                i += 1
                print("Not In Unit")
                continue
            
            accept_labels = [
                Unit.LIST,
                Unit.ITEM,
                Unit.D_CLAUSE,
                Unit.P_PHRASE
            ]

            j = i + 1
            while j < len(unit_map_vals) and unit_map_vals[j] not in units:
                if unit_map_vals[j].label_has([Unit.FRAGMENT, *accept_labels]):
                    print(f"Push to Include: {unit_map_vals[j]}")
                    bound = update_bound_fnc(bound, unit_map_vals[j])
                    break
                j += 1
                
            break

        return bound


    
    def split_tokens(self, tokens):
        verbs = [t for t in tokens if t.pos_ == "VERB"]
        print(verbs)

        # L Bound
        # It's the first token.
        l = tokens[0].i

        # M Bound
        # The L part of this bound is the first verb.
        # The R part of this bound is the last verb.
        v_l = verbs[+0].i # Do Not Ask
        v_r = verbs[-1].i

        # R Bound
        # We look for the first noun after the last verb.
        i = tokens.index(verbs[-1]) + 1
        while i < len(tokens) and tokens[i].pos_ not in ["PROPN", "NOUN", "PRON"]:
            i += 1

        # No Noun Found in R Side
        no_noun_r = i <= 0 or i >= len(tokens)

        # No Noun Found in L Side
        no_noun_l = not set([t.pos_ for t in self.main.sp_doc[l:v_l+1]]) & set(["PROPN", "NOUN", "PRON"])

        if no_noun_r and no_noun_l:
            return (-1, -1, -1, -1)
        elif no_noun_r:
            return (l, v_l, v_r, v_r)
        
        r = tokens[i].i
        if no_noun_l:
            return (v_l, v_l, v_r, r)
        else:
            return (l, v_l, v_r, r)
    
    
    
    def convert_partial_tokens(self, *, l, r):
        tokens = [self.main.sp_doc[i] for i in range(l, r+1)]
        
        v_node = Node(self.main, is_action=True)
        v_node.tokens = [token for token in tokens if token.pos_ in ["VERB", "ADV", "AUX"]]

        o_node = Node(self.main)
        o_node.tokens = [token for token in tokens if token not in v_node.tokens]

        o_node_valid = o_node.get_species() or o_node.get_traits()
        if not o_node_valid:
            return None

        event = Event(self.main)
        event.order = [v_node, o_node]

        return event
    
    
    
    def convert_tokens(self, tokens, split=None):
        tokens = sorted(tokens, key=lambda t: t.i)
        speech = [t.pos_ for t in tokens]

        # print(f"Tokens: {tokens}")
        # print(f"Speech: {speech}")

        # print(f"'VERB' not in Speech: {'VERB' not in speech}")
        # print(f"Speech[0] is ADP/SCONJ: {speech[0] in ['ADP', 'SCONJ']}")
        
        if "VERB" not in speech:
            if speech[0] in ["ADP", "SCONJ"]:
                event = Event(self.main)
                event.order = [Node(self.main, tokens=tokens)]
                # print(f"Returned Event: {event}")
                return [event]
            else:
                # print("No Verb, No ADP/SCONJ")
                return []

        if len(tokens) <= 2:
            return []
        
        l, v_l, v_r, r = split or self.split_tokens(tokens)
        
        if l == v_l == v_r == r:
            return []
        elif l == v_l:
            event = self.convert_partial_tokens(l=v_l, r=r)
            print("HELLO")
            print(event)
            return [] if not event else [event]
        elif r == v_r:
            event = self.convert_partial_tokens(l=l, r=v_r)
            print("HELLO")
            print(event)
            return [] if not event else [event]
        
        print(f"Split: ({l}, {v_l}, {v_r}, {r})")
        
        l = self.push_bound_l(l, tokens[+0].i) # Do Not Ask
        print(f"Pushed L: {l}")

        print(f"Max Bound: {tokens[-1].i}")
        r = self.push_bound_r(r, tokens[-1].i)
        print(f"Pushed R: {r}")
        
        v_node = Node(self.main, is_action=True)
        v_node.tokens = [self.main.sp_doc[i] for i in range(v_l, v_r+1)]
        print(f"v_node.tokens: {v_node.tokens}")
        
        s_node = Node(self.main)
        s_node.tokens = [self.main.sp_doc[i] for i in range(l, v_l)]
        print(f"s_node.tokens: {s_node.tokens}")
            
        o_node = Node(self.main)
        o_node.tokens = [self.main.sp_doc[i] for i in range(v_r+1, r+1)]
        print(f"o_node.tokens: {o_node.tokens}")

        s_node, v_node, o_node = self.fix_triple_direction(s_node, v_node, o_node)
        s_node, v_node, o_node = self.fix_triple_tokens(s_node, v_node, o_node)
        s_node, v_node, o_node = self.fix_triple_all_tokens(s_node, v_node, o_node)

        print("Fixed All Tokens")
        print(f"s_node.tokens: {s_node.tokens}")
        print(f"v_node.tokens: {v_node.tokens}")
        print(f"o_node.tokens: {o_node.tokens}")

        s_node_valid = s_node.get_species() or s_node.get_traits()
        o_node_valid = o_node.get_species() or o_node.get_traits()

        # print(f"s_node Valid: {s_node_valid}")
        # print(f"\ts_node Traits: {s_node.get_traits()}")
        # print(f"\ts_node Species: {s_node.get_species()}")
        
        # print(f"o_node Valid: {o_node_valid}")
        # print(f"\to_node Traits: {o_node.get_traits()}")
        # print(f"\to_node Species: {o_node.get_species()}")
        
        if not s_node_valid and not o_node_valid:
            return []
        
        event = Event(self.main)
        event.order = [s_node, v_node, o_node]

        events = [event]
        events.extend(self.convert_tokens(s_node.tokens))
        events.extend(self.convert_tokens(o_node.tokens))
        
        return [event for event in events if event]


    
    def convert_unit(self, unit):
        tokens = [*self.main.sp_doc[unit.l:unit.r+1]]
        return self.convert_tokens(tokens)


    
    def convert_svo_triple(self, triple):
        tokens = [*triple.subject, *triple.object]
        tokens = sorted(tokens, key=lambda t: t.i)
    
        l = tokens[+0].i
        v_l = triple.verb[+0].i
        v_r = triple.verb[-1].i
        r = tokens[-1].i

        tokens = [*self.main.sp_doc[l:r+1]]
        return self.convert_tokens(tokens, split=(l, v_l, v_r, r))
    
    
    
    def distinct_events(self, events):
        event_bounds_mapped = {(e.start(), e.end()): e for e in events}
        event_bounds = event_bounds_mapped.keys()   
        distinct_event_bounds = distinct_bounds(event_bounds, larger=True)    
        distinct_events = [event_bounds_mapped[b] for b in distinct_event_bounds]
        return distinct_events
    

    
    def find_events(self):
        sents = list(self.main.sp_doc.sents)
        sents_events = {sent.start: [] for sent in sents}

         # Units
        for unit_tokens in self.main.units.aggregate_units():
            print(f"Unit Tokens: {unit_tokens}")
            events = self.convert_tokens(unit_tokens)
            for event in events:
                print(f"\tEvent: {event}")
            sent_start = unit_tokens[0].sent.start
            sents_events[sent_start].extend(events)
        print()
        
        # Triples
        for triple in textacy.extract.subject_verb_object_triples(self.main.sp_doc):
            print(f"Triple: {triple}")
            events = self.convert_svo_triple(triple)
            for event in events:
                print(f"\tEvent: {event}")
            sent_start = triple.verb[0].sent.start
            sents_events[sent_start].extend(events)
        print()

        sents_events = {k: self.distinct_events(v) for k, v in sents_events.items()}
        return sents_events
    
    
    
    def overlap_in_species(self, x, y):
        sp_X = [self.main.species.span_at_token(species) for species in x.get_species()]
        sp_Y = [self.main.species.span_at_token(species) for species in y.get_species()]

        for sp_x in sp_X:
            if self.main.species.find_same_species(sp_Y, sp_x):
                return True
        return False
    
    

    def overlap_in_traits(self, x, y):
        tr_X = set([trait.lemma_.lower() for trait in x.get_traits()])
        tr_Y = set([trait.lemma_.lower() for trait in y.get_traits()])

        return tr_X & tr_Y
    

    
    def get_non_species_nouns(self, tokens):
        nouns = []
        for token in tokens:            
            if token in self.main.species.tokens:
                continue

            if token.pos_ not in ["PROPN", "NOUN"]:
                continue
            
            nouns.append(token.lower_)
            nouns.append(token.lemma_.lower())
        return nouns


    
    def overlap_in_tokens(self, x, y):
        x_tokens = self.get_non_species_nouns(x.all_tokens)
        y_tokens = self.get_non_species_nouns(y.all_tokens)
        return set(x_tokens) & set(y_tokens)
    
    
    
    def has_causal_phrase(self, x):
        speech = [token.pos_ for token in x.tokens]
        return speech[0] in ["ADP", "SCONJ"]
    
    
    
    def has_effectual_phrase(self, y):
        lower = [token.lower_ for token in y.tokens]
        return "which" in lower


    
    def has_causal_phrase_and_pron(self, n):
        speech = set([token.pos_ for token in n.tokens])
        causal_speech = speech & set(["ADP", "SCONJ"])
        pronoun_speech = speech & set(["PRON"])
        return causal_speech and pronoun_speech
    
    
    
    def can_merge_intrasent(self, X, Y):
        x = X.order[-1]
        y = Y.order[+0] # Do Not Ask

        can_merge = self.overlap_in_species(x, y)
        print(f"\tCan Merge by Species: {can_merge}")
        if can_merge:
            return (True, True)

        can_merge = self.overlap_in_tokens(x, y)
        print(f"\tCan Merge by Tokens: {can_merge}")
        if can_merge:
            return (True, True)

        can_merge = self.has_causal_phrase(x)
        print(f"\tCan Merge by Causal Phrase: {can_merge}")
        if can_merge:
            return (True, False)

        can_merge = self.has_effectual_phrase(y)
        print(f"\tCan Merge by Effectual Phrase: {can_merge}")
        if can_merge:
            return (True, True)
        
        return (False, False)
    
    

    def can_merge_intersent(self, X, Y):
        x = X.order[-1]
        y = Y.order[+0] # Do Not Ask

        can_merge = self.overlap_in_species(x, y)
        if can_merge:
            return (True, True)

        can_merge = self.overlap_in_tokens(x, y)
        if can_merge:
            return (True, True)

        can_merge = self.has_causal_phrase_and_pron(y)
        if can_merge:
            return (True, False)
        
        return (False, False)
    
    
    
    def intrasent_sequences(self, indices_1D):
        power_set_indices = power_set(indices_1D, current=[], result=[])
        sequences = []
        for subset in power_set_indices:
            if subset:
                sequences.extend(list(permutations(subset)))
        return sequences


    
    def intersent_sequences(self, indices_2D):
        sequences = interleave(indices_2D, current=[], used=[], result=[])
        return sequences
    
    
    
    def merge_intrasent(self, sent_events):
        i = 0
        while i + 1 < len(sent_events):
            X = sent_events[i]
            Y = sent_events[i+1]

            print(f"X: {X}")
            print(f"Y: {Y}")
            
            can_merge, del_end = self.can_merge_intrasent(X, Y)

            print(f"Can Merge: {can_merge}")
            
            if not can_merge:
                return None

            X.attach_event(Y, del_end=del_end)
            sent_events.pop()

        if len(sent_events[0].order) <= 2:
            return None
        return sent_events[0]
    
    
    
    def merge_intersent(self, events):
        i = 0
        while i + 1 < len(events):
            X = events[i]
            Y = events[i+1]

            can_merge, del_end = self.can_merge_intersent(X, Y)
            if not can_merge:
                return None

            X.attach_event(Y, del_end=del_end)
            events.pop()

        return events[0]
    
    
    
    def load_events(self):
        sents_events = self.find_events()

        print("Found Events:")
        for events in sents_events.values():
            for event in events:
                print(event)
        print()
            
        merged_sents_events = []
        
        for sent_events in sents_events.values():
            merged_sents_events.append([])

            # Each event can be represented with its index
            # for simplicity.
            event_indices = list(range(len(sent_events)))
            print(f"Event Indices (1D): {event_indices}")

            # The order in which you merge an event can result
            # in a different outcome. For example, E1 + E2 may
            # look different to E2 + E1, and so on.

            # Idea:
            # We try all the possible ways you can merge, which is
            # the same as trying all the different orders you can
            # merge the events. For example, merging E1, E2, and E3
            # can be done in 8 ways: E1, E2, E3, E1 + E2, E2 + E1,
            # and so on.
            
            # These are all the possible orders (or sequences) in 
            # which you can merge the above events. We will try all
            # possible sequences, adding those that work.
            sequences = self.intrasent_sequences(event_indices)
            print(f"Possible Sequences (1D): {sequences}")

            for sequence in sequences:
                events = [sent_events[i].copy() for i in sequence]
                merged_event = self.merge_intrasent(events)
                if merged_event:
                    print(f"Merged Event: {merged_event}")
                    merged_sents_events[-1].append(merged_event)
            print()
        
        # Idea:
        # Now that we've found all the events that can be made from
        # merging the events within a sentence, we can merge across
        # sentences. To do this, we also need to try all different
        # sequences of events. However, we shouldn't try and merge
        # events from the same sentence again, which means that no
        # two consecutive events in the sequence can be from the same
        # sentence. There's also another issue: we can't use a simple
        # 1D-index for each event as we're dealing with a 2D-list of events.
        # Therefore, we'll use the index equivalent to its 1D-form, or
        # something similar as we don't have a matrix.
        # If this doesn't make sense, I wouldn't be surprised, I'm more
        # of a pictures person.
        i = 0
        event_indices = []
        index_to_event = {}
        for sent_events in merged_sents_events:
            event_indices.append([])
            for event in sent_events:
                index_to_event[i] = event
                event_indices[-1].append(i)
                i += 1

        
        print(f"Event Indices (2D): {event_indices}")
        sequences = self.intersent_sequences(event_indices)
        print(f"Possible Sequences (2D): {sequences}")
        
        merged_events = []
        
        for sequence in sequences:
            events = [index_to_event[i].copy() for i in sequence]
            merged_event = self.merge_intersent(events)
            if merged_event:
                print(f"Merged Event: {merged_event}")
                merged_events.append(merged_event)
        
        return merged_events
    
    
    
    def update(self):
        self.events = self.load_events()

In [44]:
# main = Main()

In [62]:
text = "We saw that inclusion of cat resulted in the disappearance of dog."
main.update_text(text)

09/07/2025 00:10:09 - INFO - 	 Tokenize 1 inputs...
Map: 100%|██████████| 1/1 [00:00<00:00, 59.29 examples/s]
09/07/2025 00:10:16 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|██████████| 1/1 [00:05<00:00,  5.67s/it]


In [63]:
event_manager = EventManager(main)
event_manager.update()

for event in event_manager.events:
    print(event)

Unit Tokens: [We, saw]
Unit Tokens: [that, inclusion, of, cat, resulted, in, the, disappearance, of, dog, .]
[resulted]
Split: (2, 6, 6, 9)
Push L
Unit: of dog
Not In Unit
Unit: in the disappearance
Not In Unit
Unit: of cat
Not In Unit
Unit: that inclusion of cat resulted in the disappearance of dog.
Push to Include: We saw
Pushed L: 0
Max Bound: 12
Push R
Unit: We saw
Not In Unit
Unit: that inclusion of cat resulted in the disappearance of dog.
Push to Include: of cat
Pushed R: 9
v_node.tokens: [resulted]
s_node.tokens: [We, saw, that, inclusion, of, cat]
o_node.tokens: [in, the, disappearance]
Fixed All Tokens
s_node.tokens: [in, the, disappearance]
v_node.tokens: [resulted]
o_node.tokens: [We, saw, that, inclusion, of, cat]
[saw]
Split: (0, 1, 1, 3)
Push L
Unit: of dog
Not In Unit
Unit: in the disappearance
Not In Unit
Unit: of cat
Not In Unit
Unit: that inclusion of cat resulted in the disappearance of dog.
Not In Unit
Unit: We saw
Pushed L: 0
Max Bound: 5
Push R
Unit: We saw
Not I

In [64]:
main.species.tokens

[cat, dog]

In [65]:
main.species.tn_doc.ents

(cat, dog)